# Brownian Motion, the CLT, Itô Calculus, and Langevin SDEs — a pedagogical notebook

This notebook builds a coherent path from **classical calculus** and the **Central Limit Theorem (CLT)** to:

- **Brownian motion** (Wiener process) as a scaling limit of random walks,
- why Brownian paths break classical assumptions (non-differentiability, infinite variation),
- **Itô’s lemma** and the **Itô correction** (the extra \($\tfrac12 \sigma^2 f''$\) term),
- **overdamped Langevin dynamics** as *“gradient descent + noise”* and its stationary distribution.

**Dependencies:** `numpy`, `matplotlib` (standard library `math` only).  
All simulations are small-scale and designed to **verify** the mathematics visually and numerically.

---

## Notation (used consistently)

- $t$: time, $dt$: an infinitesimal time increment.
- $W_t$: standard Brownian motion (Wiener process).
- $dW_t$: Brownian increment over $[t,t+dt]$, heuristically $dW_t \sim \mathcal N(0, dt)$.
- SDE (1D):  
  $$
  dX_t = \mu(X_t,t)\,dt + \sigma(X_t,t)\,dW_t
  $$
  where $\mu$ is **drift** and $\sigma$ is **diffusion scale**.
- Langevin (1D/ \(d\)D):  
  $$
  dX_t = -\nabla U(X_t)\,dt + \sqrt{2D}\,dW_t
  $$
  where $U$ is an **energy/potential** and $D>0$ is the **diffusion coefficient**.

---

## Learning objectives (what you should be able to explain by the end)

1. How the **CLT** motivates **Gaussian increments**.
2. Why Brownian motion has **$\sqrt{dt}$** scaling.
3. Exactly why classical derivatives/chain rule fail for Brownian paths.
4. Where the $\tfrac12\sigma^2 f''$ term in **Itô’s lemma** comes from.
5. The geometric meaning of Langevin SDE as **energy descent + stochastic exploration** and why
   $$
   p_\infty(x)\propto e^{-U(x)/D}.
   $$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

# For reproducibility (feel free to change)
rng = np.random.default_rng(0)

plt.rcParams["figure.figsize"] = (8, 4.5)


## 1) Classical calculus recap: what works for smooth paths

### Limits, derivatives, Taylor expansion

If $x(t)$ is differentiable at time $t$, then for small $dt$,

$$
x(t+dt)=x(t)+x'(t)\,dt + o(dt).
$$

For a *twice differentiable* scalar function $f:\mathbb R\to\mathbb R$, Taylor’s theorem gives

$$
f(x+\Delta x)=f(x)+f'(x)\Delta x+\tfrac12 f''(x)(\Delta x)^2+o((\Delta x)^2).
$$

### Classical chain rule

If $x(t)$ is differentiable and $f$ is differentiable, then

$$
\frac{d}{dt}f(x(t)) = f'(x(t))\,x'(t).
$$

**Hidden assumption:** $x(t)$ has increments $\Delta x = x(t+dt)-x(t)$ that are order $dt$, i.e. $\Delta x = O(dt)$.
Then $(\Delta x)^2=O(dt^2)$ is negligible at first order, which is why classical calculus ignores it.

### What will break for Brownian motion?

Brownian increments scale like $\Delta W = O(\sqrt{dt})$, so $(\Delta W)^2=O(dt)$ — **not negligible**.
That single change forces a new chain rule (Itô’s lemma).


In [ ]:
# A tiny numerical reminder: for smooth x(t)=t^2, increments are O(dt)
T = 1.0
for dt in [1e-1, 1e-2, 1e-3, 1e-4]:
    t = 0.3
    dx = (t+dt)**2 - t**2
    print(f"dt={dt:>7.0e}  dx≈{dx:>10.3e}  dx/dt≈{dx/dt:>10.3e}  (dx)^2/dt≈{dx*dx/dt:>10.3e}")


In [ ]:
def normal_pdf(x, mu=0.0, sigma=1.0):
    return (1.0/(sigma*math.sqrt(2*math.pi))) * np.exp(-0.5*((x-mu)/sigma)**2)

def normal_cdf(x):
    # Standard normal CDF via erf (no scipy)
    return 0.5*(1.0 + np.vectorize(math.erf)(x/math.sqrt(2.0)))

def ks_statistic(samples):
    # One-sample KS statistic vs N(0,1)
    x = np.sort(samples)
    n = len(x)
    ecdf = np.arange(1, n+1)/n
    cdf = normal_cdf(x)
    return np.max(np.abs(ecdf - cdf))

def clt_demo(dist_name, sampler, ns=(1,2,5,10,30,100), N=20000):
    fig, axes = plt.subplots(2, 3, figsize=(12, 7))
    axes = axes.ravel()
    for i, n in enumerate(ns):
        X = sampler(size=(N, n))
        m = X.mean(axis=1)
        # normalize: sqrt(n)*(mean - E[X]) / std(X)
        EX = sampler(size=(200000,)).mean()          # quick Monte Carlo estimate for EX
        SD = sampler(size=(200000,)).std(ddof=0)     # estimate for SD
        Z = math.sqrt(n)*(m - EX)/SD
        ax = axes[i]
        ax.hist(Z, bins=60, density=True, alpha=0.7)
        xs = np.linspace(-4, 4, 400)
        ax.plot(xs, normal_pdf(xs))
        ax.set_title(f"{dist_name}, n={n}  KS≈{ks_statistic(Z):.3f}")
        ax.set_xlim(-4, 4)
    fig.suptitle("CLT: normalized sample means converge to N(0,1)")
    plt.tight_layout()
    plt.show()

# Distributions
def sample_uniform(size):     # Uniform(0,1)
    return rng.random(size=size)

def sample_bernoulli(size):   # Bernoulli(p=0.3)
    return rng.binomial(1, 0.3, size=size)

def sample_exponential(size): # Exp(rate=1)
    return rng.exponential(scale=1.0, size=size)

clt_demo("Uniform(0,1)", sample_uniform)
clt_demo("Bernoulli(0.3)", sample_bernoulli)
clt_demo("Exponential(1)", sample_exponential)


## 3) Random walk $\to$ Brownian motion: scaling and invariance intuition

### Symmetric random walk

Let $\xi_k\in\{+1,-1\}$ with $\mathbb P(\xi_k=1)=\mathbb P(\xi_k=-1)=\tfrac12$, i.i.d.
Define partial sums:

$$
S_n=\sum_{k=1}^n \xi_k.
$$

Then $\mathbb E[S_n]=0$ and $\mathrm{Var}(S_n)=n$.

### How to scale time and space

Suppose each step takes time $\Delta t$. After $n$ steps, time is $t=n\Delta t$.

If you also scale the spatial step by $\Delta x$, then position after $n$ steps is

$$
X_t = \Delta x \, S_n.
$$

Its variance is

$$
\mathrm{Var}(X_t) = (\Delta x)^2 \mathrm{Var}(S_n)
= (\Delta x)^2 n
= (\Delta x)^2 \frac{t}{\Delta t}.
$$

To obtain a **nontrivial** finite limit as $\Delta t\to 0$ (many steps in finite time), you want $\mathrm{Var}(X_t)\propto t$.
This forces

$$
(\Delta x)^2 \frac{1}{\Delta t} = \text{constant}
\quad\Longrightarrow\quad
\Delta x \propto \sqrt{\Delta t}.
$$

This is the origin of the famous **$\sqrt{dt}$** scaling.

### Diffusion coefficient $D$

A standard Brownian motion $W_t$ satisfies

$$
\mathrm{Var}(W_{t+\Delta t}-W_t)=\Delta t.
$$

A general diffusion with coefficient $D$ has

$$
\mathrm{Var}(X_{t+\Delta t}-X_t)=2D\,\Delta t.
$$

So you can model it as $X_t=\sqrt{2D}\,W_t$, i.e. scale Brownian motion by $\sqrt{2D}$.


In [ ]:
def simulate_random_walk_paths(n_steps=2000, n_paths=20, dt=1e-3):
    # +/-1 steps
    xi = rng.choice([-1, 1], size=(n_paths, n_steps))
    S = np.cumsum(xi, axis=1)
    t = np.arange(1, n_steps+1)*dt
    # Brownian scaling: dx = sqrt(dt)
    X = np.sqrt(dt) * S
    return t, X

t, X = simulate_random_walk_paths(n_steps=3000, n_paths=30, dt=1e-3)

plt.figure()
for i in range(X.shape[0]):
    plt.plot(t, X[i], alpha=0.7)
plt.title("Rescaled random walks approximate Brownian motion (many sample paths)")
plt.xlabel("t")
plt.ylabel("X_t")
plt.show()


## 4) Brownian motion properties: increments, variance, non-differentiability, quadratic variation

### Definition (standard Brownian motion)

A process $(W_t)_{t\ge0}$ is a standard Brownian motion if:

1. $W_0=0$ a.s.
2. **Independent increments:** $W_{t_2}-W_{t_1}$ is independent of the past for $t_2>t_1$.
3. **Gaussian increments:** $W_{t+\Delta t}-W_t\sim\mathcal N(0,\Delta t)$.
4. **Continuous paths:** $t\mapsto W_t$ is continuous a.s.

### Non-differentiability in one diagnostic

A key signature is the scaling:

$$
\frac{W_{t+dt}-W_t}{dt} \sim \mathcal N\left(0,\frac{1}{dt}\right),
$$

whose variance blows up as $dt\to 0$. So the “velocity” becomes unbounded — classical derivatives fail.

### Quadratic variation

For a partition $0=t_0<t_1<\dots<t_N=T$, define

$$
[W]_T^{(N)} := \sum_{k=0}^{N-1} (W_{t_{k+1}}-W_{t_k})^2.
$$

A fundamental theorem is that as the mesh $\max_k(t_{k+1}-t_k)\to 0$,

$$
[W]_T^{(N)} \to T \quad\text{(in probability, and almost surely along refinements)}.
$$

This *finite, nonzero quadratic variation* is exactly why $(dW_t)^2$ behaves like $dt$ in Itô calculus.


In [ ]:
def simulate_brownian(T=1.0, dt=1e-3, n_paths=500):
    n = int(T/dt)
    dW = math.sqrt(dt) * rng.standard_normal(size=(n_paths, n))
    W = np.cumsum(dW, axis=1)
    t = np.arange(1, n+1)*dt
    return t, W, dW

T = 1.0
dt = 1e-3
t, W, dW = simulate_brownian(T=T, dt=dt, n_paths=200)

# (a) Increments look Gaussian with variance dt
inc = dW[:, 0]  # increments over the first dt
print("Empirical mean(dW) ≈", inc.mean(), "  empirical var(dW) ≈", inc.var(), "  target var =", dt)

plt.figure()
plt.hist(inc/ math.sqrt(dt), bins=60, density=True, alpha=0.7)
xs = np.linspace(-4,4,400)
plt.plot(xs, normal_pdf(xs))
plt.title("Standardized increments dW/sqrt(dt) ≈ N(0,1)")
plt.xlabel("z")
plt.ylabel("density")
plt.show()

# (b) Quadratic variation check
qv = np.sum(dW**2, axis=1)  # sum of squared increments over [0,T]
print("Quadratic variation: mean ≈", qv.mean(), "  std ≈", qv.std(), "  target =", T)

plt.figure()
plt.hist(qv, bins=60, density=True, alpha=0.7)
plt.title("Quadratic variation estimates across paths (should concentrate near T)")
plt.xlabel("sum (dW)^2")
plt.ylabel("density")
plt.show()


### Zoom-in self-similarity and non-differentiability (visual)

A differentiable curve “looks straight” when you zoom in enough. A Brownian path never settles into a straight line; it remains jagged at every scale.
We'll plot a single Brownian path and zoom into a small interval.


In [ ]:
t, W1, _ = simulate_brownian(T=1.0, dt=1e-4, n_paths=1)
W1 = W1[0]

plt.figure()
plt.plot(t, W1)
plt.title("One Brownian path over [0,1]")
plt.xlabel("t")
plt.ylabel("W_t")
plt.show()

# Zoom into a short interval
a, b = 0.40, 0.42
mask = (t >= a) & (t <= b)

plt.figure()
plt.plot(t[mask], W1[mask])
plt.title(f"Zoom-in: Brownian path on [{a},{b}] (still jagged)")
plt.xlabel("t")
plt.ylabel("W_t")
plt.show()


## 5) Why the classical chain rule changes: \(dW_t^2 \sim dt\)

Consider an Itô process

\[
dX_t = \mu(X_t,t)\,dt + \sigma(X_t,t)\,dW_t.
\]

Over a small interval \(dt\), the increment is

\[
\Delta X \approx \mu\,dt + \sigma\,\Delta W,
\quad\text{where}\quad
\Delta W \sim \mathcal N(0,dt).
\]

Key scaling:

- \(\Delta W = O(\sqrt{dt})\)
- so \((\Delta W)^2 = O(dt)\)
- while \(dt\cdot \Delta W = O(dt^{3/2})\) (negligible vs \(dt\))
- and \(dt^2 = O(dt^2)\) (also negligible)

### Derivation sketch from Taylor expansion

Let \(f\) be \(C^2\). Taylor expand to second order:

\[
f(X_{t+dt})-f(X_t)
\approx f'(X_t)\Delta X + \tfrac12 f''(X_t) (\Delta X)^2.
\]

Now expand \((\Delta X)^2\):

\[
(\Delta X)^2 \approx (\mu dt + \sigma \Delta W)^2
= \mu^2 dt^2 + 2\mu\sigma\,dt\,\Delta W + \sigma^2 (\Delta W)^2.
\]

As \(dt\to 0\), the first two terms vanish faster than \(dt\), but the last term is order \(dt\).
Thus **the second-order Taylor term contributes at first order in time**:

\[
\tfrac12 f''(X_t)(\Delta X)^2 \approx \tfrac12 f''(X_t)\sigma^2 (\Delta W)^2 \approx \tfrac12 f''(X_t)\sigma^2 dt.
\]

That is the “Itô correction” and the precise reason Itô calculus differs from classical calculus.

We'll verify the scaling numerically next.


In [ ]:
# Verify scaling: for Brownian increments, (dW)^2 behaves like dt in expectation and concentrates as dt->0.

def scaling_experiment(dts=(1e-1, 5e-2, 1e-2, 5e-3, 1e-3, 5e-4), N=200000):
    rows = []
    for dt in dts:
        dW = math.sqrt(dt) * rng.standard_normal(size=N)
        rows.append((dt,
                     dW.mean(),
                     dW.var(),
                     (dW**2).mean(),
                     (dW**2).var(),
                     np.mean(np.abs(dW))/math.sqrt(dt)))
    return rows

rows = scaling_experiment()
print("dt      E[dW]      Var[dW]     E[(dW)^2]  Var[(dW)^2]  E|dW|/sqrt(dt)")
for r in rows:
    print(f"{r[0]:.0e}  {r[1]: .3e}  {r[2]: .3e}  {r[3]: .3e}  {r[4]: .3e}  {r[5]: .3f}")


## 6) Itô’s lemma: derivation, examples, and numerical verification

### Derivation (1D)

Let \(X_t\) satisfy

\[
dX_t = \mu(X_t,t)\,dt + \sigma(X_t,t)\,dW_t.
\]

Let \(f(x,t)\) be \(C^{2,1}\): twice differentiable in \(x\), once in \(t\).  
Do a second-order Taylor expansion in both variables:

\[
df = f(X_{t+dt},t+dt)-f(X_t,t)
\approx f_t\,dt + f_x\,\Delta X + \tfrac12 f_{xx}(\Delta X)^2,
\]

where derivatives are evaluated at \((X_t,t)\) and \(\Delta X = \mu dt + \sigma \Delta W\).

Using the scaling from the previous section:

- \((\Delta W)^2 \approx dt\),
- \(dt\Delta W\) and \(dt^2\) are negligible.

So

\[
(\Delta X)^2 \approx \sigma^2 (\Delta W)^2 \approx \sigma^2 dt.
\]

Substitute:

\[
df \approx \left(f_t + \mu f_x + \tfrac12 \sigma^2 f_{xx}\right)dt + \sigma f_x\,dW_t.
\]

This is **Itô’s lemma**:

\[
\boxed{
df(X_t,t) =
\left(\partial_t f + \mu\,\partial_x f + \tfrac12\sigma^2\,\partial_{xx}f\right)dt
+ \sigma\,\partial_x f\, dW_t.
}
\]

---

### Worked example 1: \(f(x)=x^2\) for Brownian motion

Take \(X_t=W_t\), so \(\mu=0,\sigma=1\).  
Then \(f_x=2x,\ f_{xx}=2\). Itô gives:

\[
d(W_t^2) = 2W_t\,dW_t + 1\cdot dt.
\]

Integrating from \(0\) to \(T\):

\[
W_T^2 - W_0^2 = \int_0^T 2W_t\,dW_t + \int_0^T dt.
\]

So
\[
W_T^2 - T
\]
is a martingale (mean 0), and \(\mathbb E[W_T^2]=T\).

### Worked example 2: \(f(x)=e^{ax}\) for Brownian motion

Again \(X_t=W_t\), \(\mu=0,\sigma=1\).
Here \(f_x = a e^{ax},\ f_{xx}=a^2 e^{ax}\). Then

\[
d(e^{aW_t}) = a e^{aW_t}\,dW_t + \tfrac12 a^2 e^{aW_t}\,dt.
\]

A useful corollary is that
\[
M_t = \exp\left(aW_t - \tfrac12 a^2 t\right)
\]
is a martingale.

### Worked example 3 (time-dependent): \(f(x,t)=\sin(x)\,e^{-t/2}\) with \(X_t=W_t\)

Compute derivatives:
- \(f_t = -\tfrac12 \sin(x)e^{-t/2}\),
- \(f_x = \cos(x)e^{-t/2}\),
- \(f_{xx} = -\sin(x)e^{-t/2}\).

Plug into Itô with \(\mu=0,\sigma=1\):

\[
df = \left(f_t + \tfrac12 f_{xx}\right)dt + f_x dW_t
= \left(-\tfrac12\sin(x)e^{-t/2} + \tfrac12(-\sin(x)e^{-t/2})\right)dt + \cos(x)e^{-t/2}\,dW_t
= -\sin(x)e^{-t/2}\,dt + \cos(x)e^{-t/2}\,dW_t.
\]

So the drift term is \(-\sin(W_t)e^{-t/2}\).


In [ ]:
def euler_maruyama(mu, sigma, x0, T, dt):
    n = int(T/dt)
    t = np.arange(0, n+1)*dt
    X = np.empty(n+1)
    X[0] = x0
    dW = math.sqrt(dt)*rng.standard_normal(size=n)
    for k in range(n):
        tk = t[k]
        xk = X[k]
        X[k+1] = xk + mu(xk, tk)*dt + sigma(xk, tk)*dW[k]
    return t, X, dW

def ito_verify(f, fx, fxx, ft, mu, sigma, x0=0.0, T=1.0, dt=1e-3):
    t, X, dW = euler_maruyama(mu, sigma, x0, T, dt)
    # LHS
    lhs = f(X[-1], T) - f(X[0], 0.0)
    # RHS (discretized)
    drift = 0.0
    mart = 0.0
    for k in range(len(dW)):
        tk = t[k]
        xk = X[k]
        drift += (ft(xk, tk) + mu(xk, tk)*fx(xk, tk) + 0.5*(sigma(xk, tk)**2)*fxx(xk, tk))*dt
        mart  += sigma(xk, tk)*fx(xk, tk)*dW[k]
    rhs = drift + mart
    return lhs, rhs, lhs-rhs

# Example 1: f(x)=x^2, X=W (mu=0, sigma=1)
mu0 = lambda x,t: 0.0
sig1 = lambda x,t: 1.0

f1   = lambda x,t: x**2
fx1  = lambda x,t: 2*x
fxx1 = lambda x,t: 2.0
ft1  = lambda x,t: 0.0

# Run multiple paths and check error vs dt
def error_vs_dt(dts, n_paths=200):
    errs = []
    for dt in dts:
        e = []
        for _ in range(n_paths):
            lhs, rhs, err = ito_verify(f1, fx1, fxx1, ft1, mu0, sig1, x0=0.0, T=1.0, dt=dt)
            e.append(err)
        errs.append((dt, float(np.mean(np.abs(e))), float(np.std(e))))
    return errs

dts = [1e-1, 5e-2, 2e-2, 1e-2, 5e-3, 2e-3, 1e-3]
errs = error_vs_dt(dts, n_paths=300)

print("dt      mean|error|   std(error)")
for dt, m, s in errs:
    print(f"{dt:>7.0e}   {m:>10.3e}   {s:>10.3e}")

plt.figure()
plt.plot([e[0] for e in errs], [e[1] for e in errs], marker="o")
plt.xscale("log")
plt.yscale("log")
plt.title("Itô verification for f(x)=x^2, X=W: error decreases as dt decreases")
plt.xlabel("dt")
plt.ylabel("mean absolute error")
plt.show()


In [ ]:
# Example 2: f(x)=exp(a x), X=W
a = 0.8
f2   = lambda x,t: math.exp(a*x)
fx2  = lambda x,t: a*math.exp(a*x)
fxx2 = lambda x,t: (a*a)*math.exp(a*x)
ft2  = lambda x,t: 0.0

# Single-path demonstration: compare both sides
lhs, rhs, err = ito_verify(f2, fx2, fxx2, ft2, mu0, sig1, x0=0.0, T=1.0, dt=1e-4)
print("Example 2 (single path): LHS, RHS, error =", lhs, rhs, err)

# Many paths: error statistics
errs = []
for _ in range(500):
    _, _, e = ito_verify(f2, fx2, fxx2, ft2, mu0, sig1, x0=0.0, T=1.0, dt=1e-3)
    errs.append(e)
errs = np.array(errs)
print("Example 2: mean error =", errs.mean(), "  mean |error| =", np.mean(np.abs(errs)))


In [ ]:
# Example 3: time-dependent f(x,t)=sin(x)*exp(-t/2), X=W
f3   = lambda x,t: math.sin(x)*math.exp(-t/2.0)
fx3  = lambda x,t: math.cos(x)*math.exp(-t/2.0)
fxx3 = lambda x,t: -math.sin(x)*math.exp(-t/2.0)
ft3  = lambda x,t: -0.5*math.sin(x)*math.exp(-t/2.0)

lhs, rhs, err = ito_verify(f3, fx3, fxx3, ft3, mu0, sig1, x0=0.0, T=1.0, dt=1e-4)
print("Example 3 (single path): error =", err)

# Error vs dt
dts = [5e-2, 2e-2, 1e-2, 5e-3, 2e-3, 1e-3]
errs = []
for dt in dts:
    e = []
    for _ in range(300):
        _, _, ee = ito_verify(f3, fx3, fxx3, ft3, mu0, sig1, x0=0.0, T=1.0, dt=dt)
        e.append(ee)
    errs.append((dt, float(np.mean(np.abs(e)))))
plt.figure()
plt.plot([d for d,_ in errs], [m for _,m in errs], marker="o")
plt.xscale("log"); plt.yscale("log")
plt.title("Itô verification for time-dependent example: error vs dt")
plt.xlabel("dt"); plt.ylabel("mean |error|")
plt.show()


## 7) From Itô to Langevin: “gradient flow + noise” and equilibrium density

### Overdamped Langevin dynamics

In \(d\) dimensions:

\[
dX_t = -\nabla U(X_t)\,dt + \sqrt{2D}\,dW_t,
\]

where \(W_t\) is \(d\)-dimensional Brownian motion (independent components).

**Interpretation:**
- Drift \(-\nabla U\): a vector field pointing “downhill” in the energy landscape \(U\).  
  This is precisely the same direction used by gradient descent.
- Diffusion \(\sqrt{2D}\,dW_t\): isotropic random perturbations that explore space.
- The competition yields sampling concentrated in low-\(U\) regions but still able to cross barriers.

### Fokker–Planck derivation of the stationary distribution

Let \(p(x,t)\) be the probability density of \(X_t\).
For an Itô SDE

\[
dX_t = b(X_t)\,dt + \sqrt{2D}\,dW_t
\]
(with constant isotropic diffusion), the Fokker–Planck equation is

\[
\partial_t p = -\nabla\cdot(b\,p) + D\,\Delta p.
\]

For Langevin, \(b(x)=-\nabla U(x)\), so

\[
\partial_t p = \nabla\cdot\left(p\,\nabla U\right) + D\,\Delta p.
\]

A stationary density \(p_\infty\) satisfies \(\partial_t p_\infty=0\):

\[
0 = \nabla\cdot\left(p_\infty \nabla U\right) + D\,\Delta p_\infty
= \nabla\cdot\left(p_\infty \nabla U + D\,\nabla p_\infty\right).
\]

A sufficient (and standard) way to solve this is to set the probability **current** to zero:

\[
J(x) := -b(x)p_\infty(x) - D\nabla p_\infty(x) = 0.
\]

Since \(b=-\nabla U\), this condition becomes

\[
\nabla U\,p_\infty + D\nabla p_\infty = 0
\quad\Longrightarrow\quad
\nabla p_\infty = -\frac{1}{D}p_\infty \nabla U.
\]

Divide by \(p_\infty>0\):

\[
\nabla \log p_\infty = -\frac{1}{D}\nabla U
\quad\Longrightarrow\quad
\log p_\infty = -\frac{1}{D}U + C
\]

so

\[
\boxed{p_\infty(x) \propto e^{-U(x)/D}.}
\]

This is a Boltzmann/Gibbs density with “temperature” proportional to \(D\).


## 8) Geometric visuals: double-well Langevin in 1D

We use a classic **double-well** potential:

\[
U(x) = \frac{1}{4}(x^2-1)^2.
\]

- Minima near \(x=\pm 1\) (two wells),
- A barrier at \(x=0\).

The Langevin SDE is

\[
dX_t = -U'(X_t)\,dt + \sqrt{2D}\,dW_t.
\]

We will:
1. Simulate trajectories and show **barrier crossing**.
2. Compare the empirical histogram to the theoretical stationary density \(p_\infty(x)\propto e^{-U(x)/D}\).
3. Visualize the drift field \(-U'(x)\) as a geometric vector field (in 1D: arrows along the line).

### Sensible defaults
- \(D=0.2\) (enough noise to cross the barrier sometimes),
- small time step (Euler–Maruyama),
- long run with a burn-in to approximate stationarity.


In [ ]:
def U(x):
    return 0.25*(x*x - 1.0)**2

def dU(x):
    # derivative of 0.25*(x^2-1)^2 is x*(x^2-1)
    return x*(x*x - 1.0)

def langevin_1d(x0=0.0, T=50.0, dt=1e-3, D=0.2):
    n = int(T/dt)
    t = np.arange(0, n+1)*dt
    X = np.empty(n+1)
    X[0] = x0
    dW = math.sqrt(dt) * rng.standard_normal(size=n)
    noise_scale = math.sqrt(2.0*D)
    for k in range(n):
        xk = X[k]
        X[k+1] = xk - dU(xk)*dt + noise_scale*dW[k]
    return t, X

# Simulate a few trajectories
D = 0.2
dt = 2e-3
T = 25.0

plt.figure()
for x0 in [-1.0, -0.2, 0.2, 1.0]:
    t, X = langevin_1d(x0=x0, T=T, dt=dt, D=D)
    plt.plot(t, X, alpha=0.8, label=f"x0={x0}")
plt.title("Langevin trajectories in a double well (barrier crossing may occur)")
plt.xlabel("t")
plt.ylabel("X_t")
plt.legend()
plt.show()


In [ ]:
# Drift field visualization: -U'(x)
xs = np.linspace(-2.0, 2.0, 200)
drift = -dU(xs)

plt.figure()
plt.plot(xs, drift)
plt.axhline(0.0)
plt.title("Drift field b(x) = -U'(x): arrows point downhill in U")
plt.xlabel("x")
plt.ylabel("b(x)")
plt.show()

plt.figure()
plt.plot(xs, U(xs))
plt.title("Double-well potential U(x)")
plt.xlabel("x")
plt.ylabel("U(x)")
plt.show()


In [ ]:
# Long run to estimate stationary density
D = 0.2
dt = 1e-3
T = 200.0

t, X = langevin_1d(x0=0.0, T=T, dt=dt, D=D)

burn_in = int(0.2*len(X))
samples = X[burn_in:]  # approximate stationarity samples

# Empirical histogram
plt.figure()
counts, bins, _ = plt.hist(samples, bins=120, density=True, alpha=0.7, label="empirical")

# Theoretical stationary density proportional to exp(-U/D)
grid = np.linspace(-2.2, 2.2, 800)
p_unnorm = np.exp(-U(grid)/D)
# Normalize numerically by trapezoid rule
Z = np.trapz(p_unnorm, grid)
p = p_unnorm / Z
plt.plot(grid, p, label=r"$\propto e^{-U(x)/D}$ (normalized)")

plt.title(f"Stationary density check (D={D})")
plt.xlabel("x")
plt.ylabel("density")
plt.legend()
plt.show()

print("Empirical mean:", samples.mean(), "Empirical var:", samples.var())


### Optional: “density over time” as a geometric evolution (coarse)

The Fokker–Planck equation describes how an initial distribution flows under:
- deterministic transport by drift (downhill flow),
- plus diffusion (spreading).

We can visualize this coarsely by histogram snapshots over time.


In [ ]:
# Coarse density evolution snapshots
D = 0.2
dt = 1e-3
T = 20.0
t, X = langevin_1d(x0=1.5, T=T, dt=dt, D=D)  # start away from wells

snap_times = [0.2, 1.0, 3.0, 8.0, 15.0, 20.0]
snap_idx = [int(st/dt) for st in snap_times]
bins = np.linspace(-2.2, 2.2, 120)

plt.figure(figsize=(10,6))
for st, idx in zip(snap_times, snap_idx):
    window = X[max(0, idx-2000):idx+1]  # local window as a proxy for "distribution"
    hist, edges = np.histogram(window, bins=bins, density=True)
    centers = 0.5*(edges[:-1]+edges[1:])
    plt.plot(centers, hist, alpha=0.8, label=f"t≈{st}")
plt.plot(grid, p, label="stationary (theory)")
plt.title("Coarse density snapshots (using sliding-window histograms)")
plt.xlabel("x")
plt.ylabel("density")
plt.legend()
plt.show()


## 2) The Central Limit Theorem (CLT): derivation outline + simulation

### Statement (classical i.i.d. CLT)

Let $(X_k)_{k\ge1}$ be i.i.d. with mean \(\mathbb E[X_k]=m\) and variance \(\mathrm{Var}(X_k)=s^2\in(0,\infty)\).
Define the normalized sum

$$
Z_n=\frac{\sum_{k=1}^n X_k - n m}{s\sqrt{n}}.
$$

Then as \(n\to\infty\), \(Z_n \Rightarrow \mathcal N(0,1)\) (convergence in distribution).

### A clean derivation outline using moment generating functions (MGFs)

Let \(Y_k = \frac{X_k-m}{s}\) so that \(\mathbb E[Y_k]=0\), \(\mathrm{Var}(Y_k)=1\).
Then

\[
Z_n = \frac{1}{\sqrt n}\sum_{k=1}^n Y_k.
\]

Let \(M_Y(t)=\mathbb E[e^{tY}]\) be the MGF of \(Y\), assumed finite near \(t=0\).
Because \(\mathbb E[Y]=0\) and \(\mathbb E[Y^2]=1\), we have a Taylor expansion around \(0\):

\[
M_Y(t)=1+\frac{t^2}{2}+o(t^2).
\]

Now compute the MGF of \(Z_n\):

\[
M_{Z_n}(t)=\mathbb E\left[e^{t Z_n}\right]
=\mathbb E\left[\exp\left(\frac{t}{\sqrt n}\sum_{k=1}^n Y_k\right)\right]
=\prod_{k=1}^n \mathbb E\left[e^{\frac{t}{\sqrt n} Y_k}\right]
=\left(M_Y\left(\frac{t}{\sqrt n}\right)\right)^n.
\]

Using the expansion \(M_Y(u)=1+\tfrac{u^2}{2}+o(u^2)\) with \(u=t/\sqrt n\):

\[
M_Y\left(\frac{t}{\sqrt n}\right)=1+\frac{t^2}{2n}+o\left(\frac{1}{n}\right).
\]

Therefore

\[
M_{Z_n}(t)=\left(1+\frac{t^2}{2n}+o\left(\frac{1}{n}\right)\right)^n \to e^{t^2/2},
\]

which is precisely the MGF of \(\mathcal N(0,1)\). This proves the CLT under the stated conditions.

### Why this motivates Gaussian increments

If you add many tiny, weakly-dependent fluctuations, their aggregate (after proper normalization) becomes Gaussian.
A *random walk* is literally a sum of i.i.d. steps, so its *macroscopic increments* become Gaussian — the core reason Brownian motion has Gaussian increments.


## 9) Summary: what to remember + common pitfalls

### What to remember

- **CLT**: sums of many small independent fluctuations become Gaussian after normalization.  
  This is the fundamental reason **Gaussian increments** arise in diffusion limits.

- **Random walk scaling**: to get a nontrivial continuous-time limit, you must scale steps as  
  \[
  \Delta x \propto \sqrt{\Delta t}.
  \]
  That is exactly Brownian scaling and leads to \(\mathrm{Var}(\Delta W)=\Delta t\).

- **What breaks from classical calculus**:
  Brownian paths are continuous but **almost surely nowhere differentiable**, with **finite nonzero quadratic variation**:
  \[
  \sum ( \Delta W )^2 \to T.
  \]

- **Itô’s lemma**:
  because \((dW_t)^2\) behaves like \(dt\), the second-order Taylor term contributes at first order:
  \[
  df = \left(f_t + \mu f_x + \tfrac12 \sigma^2 f_{xx}\right)dt + \sigma f_x dW_t.
  \]
  That \(\tfrac12\sigma^2 f_{xx}\) is the **Itô correction**.

- **Langevin geometry**:
  \[
  dX_t = -\nabla U(X_t)\,dt + \sqrt{2D}\,dW_t
  \]
  is **gradient descent** in the energy landscape plus isotropic exploration.
  The stationary distribution is
  \[
  p_\infty(x)\propto e^{-U(x)/D}.
  \]

### Common pitfalls

1. Treating \(dW_t\) like \(dt\): **wrong scale**. Remember \(dW_t=O(\sqrt{dt})\).
2. Dropping \((dW_t)^2\): **cannot** — it contributes at order \(dt\).
3. Confusing diffusion coefficient: in \(dX_t=\sqrt{2D}\,dW_t\), variance over \(\Delta t\) is \(2D\Delta t\).
4. Euler–Maruyama accuracy: decreasing \(dt\) improves Itô-identity checks; coarse \(dt\) creates discretization error.

---

### Where this connects to diffusion-based generative modeling (one line)

In score-based and diffusion generative models, reverse-time sampling is a controlled SDE/ODE whose noise structure is inherited from Brownian motion; understanding **Itô vs classical calculus** is what makes those derivations correct.
